In [1]:
#importando addins necessarios

import pandas as pd
import pageviewapi
from datetime import datetime, timedelta

#Importando lista de especies e escolhendo colunas de interesse#

columns_to_keep = ["nome_popular","order","family","species","author_name"]


rodentia_sp = pd.read_csv ("Scopus_Rodentia.csv", sep =",", usecols=columns_to_keep)


#retirando espaco e incluindo underline como separador

col= "species"

rodentia_sp[col]= rodentia_sp[col].str.rstrip().str.replace(" ", "_")


#Fazendo uma lista com as especies e definindo as datas

R_lista_sp= rodentia_sp["species"].tolist()

In [6]:
#Criando codigo para Pageviews
data_inicio = "20150701"
data_fim = datetime.now().strftime("%Y%m%d")

def puxar_pageviews_annual(lista_sp, start_date, end_date):
    """
    Pulls annual Wikipedia pageviews for each species.
    Retries species with no data found.
    """
    pageviews_list = []
    retry_list = []
    
    # First attempt
    for sp in lista_sp:
        try:
            views = pageviewapi.per_article('en.wikipedia', sp, start=start_date, end=end_date, access='all-access', agent='all-agents', granularity='monthly')
            
            if views['items']:
                # Aggregate by year
                annual_views = {}
                for item in views['items']:
                    year = item['timestamp'][:4]
                    annual_views[year] = annual_views.get(year, 0) + item['views']
                
                for year, total_views in annual_views.items():
                    pageviews_list.append({'species': sp, 'year': int(year), 'views': total_views})
            else:
                retry_list.append(sp)
        except:
            retry_list.append(sp)
    
    # Retry species with no data
    for sp in retry_list:
        try:
            views = pageviewapi.per_article('pt.wikipedia', sp, start='20150701', end=end_date, access='all-access', agent='all-agents', granularity='monthly')
            if views['items']:
                annual_views = {}
                for item in views['items']:
                    year = item['timestamp'][:4]
                    annual_views[year] = annual_views.get(year, 0) + item['views']
                
                for year, total_views in annual_views.items():
                    pageviews_list.append({'species': sp, 'year': int(year), 'views': total_views})
        except:
            pass
    
    return pd.DataFrame(pageviews_list)
Rodentia_pageviews_df = puxar_pageviews_annual(R_lista_sp, data_inicio, data_fim)
print(Rodentia_pageviews_df)

                species  year  views
0          Galea_spixii  2015    332
1          Galea_spixii  2016    777
2          Galea_spixii  2017    626
3          Galea_spixii  2018    612
4          Galea_spixii  2019    557
..                  ...   ...    ...
380  Trinomys_yonenagae  2026     22
381    Calomys_mattevii  2023     55
382    Calomys_mattevii  2024    188
383    Calomys_mattevii  2025    187
384    Calomys_mattevii  2026     12

[385 rows x 3 columns]


In [8]:
#Criando tabela completa com todas as espécies e anos
# Create all combinations of species and years
complete_df = pd.DataFrame([(sp, year) for sp in R_lista_sp for year in range(2015, 2027)], 
                           columns=['species', 'year'])

# Merge with actual data
complete_df = complete_df.merge(Rodentia_pageviews_df, on=['species', 'year'], how='left')

# Identify species with no data
species_no_data = [sp for sp in R_lista_sp if sp not in Rodentia_pageviews_df['species'].unique()]

# Fill values: N/A for no page, -- for no views that year
complete_df['views'] = complete_df.apply(
    lambda row: 'N/A' if row['species'] in species_no_data 
    else ('--' if pd.isna(row['views']) else int(row['views'])), axis=1)

# Sort and save
complete_df = complete_df.sort_values(['species', 'year']).reset_index(drop=True)
complete_df.to_csv('rodentia_pageviews_annual.csv', index=False)
print(f"Arquivo salvo: rodentia_pageviews_annual.csv\nTotal registros: {len(complete_df)}")

Arquivo salvo: rodentia_pageviews_annual.csv
Total registros: 420
